In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.filters import Q500US
import numpy as np
import pandas as pd 
import datetime 

from __future__ import division

from quantopian.pipeline.experimental import QTradableStocksUS

## High Low Ratio 

+ 過去２５２日（一年）で、今日の価格が一番高い銘柄数と今日の価格が一番安い銘柄数の割合を出したもの
+ 詳しくはこのあたりを見てください。[New 52-week Highs and Lows [ChartSchool]](https://stockcharts.com/school/doku.php?id=chart_school:index_symbols:52_week_high_low)
+ 数字が高ければ、市場がイケイケドンドン＼(^o^)／
+ 数字が低ければ、(((( ;ﾟдﾟ))))ｱﾜﾜﾜﾜ
+ と考えることができるかもしれない、という指標です

## QTradableStocksUS

銘柄はQuantopianが作った `QTradableStocksUS()` を使います。詳しくは、[こちら](https://www.quantopian.com/help#built-in-filters)





In [ ]:
class RollingMax(CustomFactor):
    inputs = [USEquityPricing.high]
    
    def compute(self, today, asset_ids, out, values): 
        out[:] = np.nanmax(values,axis=0) 

class RollingMin(CustomFactor):
    inputs = [USEquityPricing.low]
    def compute(self, today, asset_ids, out, values): 
        out[:] = np.nanmin(values,axis=0) 

class isHighToday(CustomFactor):
    inputs = [USEquityPricing.high]
    window_length = 252
    def compute(self, today, asset_ids, out, values): 
        max_values = np.nanmax(values,axis=0)
        high_252 = max_values  == values[-1]
        out[:] =  high_252
        
class isLowToday(CustomFactor):
    inputs = [USEquityPricing.low]
    window_length = 252
    def compute(self, today, asset_ids, out, values): 
        min_values = np.nanmin(values,axis=0)
        low_252 = min_values == values[-1]
        out[:] = low_252
        
        
    
def make_pipeline1():
    base_universe = QTradableStocksUS()
    pipe = Pipeline()
    pipe.add(isHighToday(), "isHighToday")
    pipe.add(isLowToday(), "isLowToday")
    pipe.set_screen(base_universe)
    return pipe
    
    
        
def make_pipeline():
    #base_universe = Q500US()
    base_universe = QTradableStocksUS()
    high = USEquityPricing.high
    low = USEquityPricing.low
    high_252 = RollingMax(window_length=252)
    low_252 = RollingMin(window_length=252)

    is_max = high_252.eq(high)
    is_min = low_252.eq(low)
    
    pipe = Pipeline(
        columns = {
            'close': USEquityPricing.close.latest,
            'high': high.latest,
            'low': low.latest,
            'high_252': high_252,
            'low_252': low_252,
            'hit_new_high': is_max, 
            'hit_new_low': is_min, 
        
        },
        screen = base_universe)
    return pipe 


end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)
results = run_pipeline(make_pipeline1(), 
                       start_date=start_date, 
                       end_date = end_date )


In [ ]:
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor
from quantopian.research import run_pipeline

import numpy as np

my_window_length = 252

class isHighToday(CustomFactor):
    inputs = [USEquityPricing.high]
    window_length = my_window_length
    def compute(self, today, asset_ids, out, values): 
        max_values = np.nanmax(values,axis=0)
        high_252 = max_values  == values[-1]
        out[:] =  high_252
        
class isLowToday(CustomFactor):
    inputs = [USEquityPricing.low]
    window_length = my_window_length
    def compute(self, today, asset_ids, out, values): 
        min_values = np.nanmin(values,axis=0)
        low_252 = min_values == values[-1]
        out[:] = low_252
        

        
def make_pipeline1():
    base_universe = QTradableStocksUS()
    pipe = Pipeline()
    pipe.add(isHighToday(), "isHighToday")
    pipe.add(isLowToday(), "isLowToday")
    pipe.set_screen(base_universe)
    return pipe


end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)
results = run_pipeline(make_pipeline1(), 
                       start_date=start_date, 
                       end_date = end_date )

highlows = results.groupby(level=0)[["isHighToday", "isLowToday"]].sum()
highlows["high_low_ratio"] = highlows["isHighToday"] / ( highlows["isHighToday"] +  highlows["isLowToday"])
highlows["high_low_ratio_ma10"] = highlows["high_low_ratio"].rolling(10).mean()

spy = get_pricing("SPY", start_date=start_date,end_date= end_date, fields='price')
highlows["spy"] = spy
highlows[["high_low_ratio_ma10", "spy"]].plot(secondary_y = "spy", linewidth=5)

In [ ]:
start_date = '2014-01-01'
spy = get_pricing("SPY", start_date=start_date,end_date= datetime.datetime.today())
end_date = spy.dropna().index[-1]

results = run_pipeline(make_pipeline(), 
                       start_date=start_date, 
                       end_date = end_date )

l = list()
dates = results.index.get_level_values(0).unique()

for date in dates:
    print(date)
    df = results.ix[date]
    highs = df[df["hit_new_high"] == True]["hit_new_high"].sum()   
    lows  = df[df["hit_new_low"] == True]["hit_new_low"].sum()
    
    l.append({"highs":highs, "lows":lows, "HLRatio": highs * 1.0 / (highs + lows) })

df = pd.DataFrame(l, index=dates)


pd.concat([spy, pd.rolling_mean(df.HLRatio, 10)], axis=1)[["price", "HLRatio"]].dropna().plot(secondary_y="price")
print (datetime.datetime.now())



In [ ]:

ax = pd.concat([spy, pd.rolling_mean(df.HLRatio, 10)], axis=1)[["price", "HLRatio"]].dropna().ix["2016-01-01":]
ax = ax.plot(secondary_y="price",linewidth = 5.0 )
todayis = df.index[-1].strftime("%Y-%m-%d")
hltoday = pd.rolling_mean(df.HLRatio, 10).ix[-1]
ax.set_title("HLRatio %s %s" % (todayis, hltoday))






## 月間フィントーク
https://fintalk.connpass.com/